In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import json
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기

---
### 서울 열린데이터 광장 
### API 활용하여 데이터 수집하기

In [2]:
# 서울 각 지역별 API url과 인증키 데이터가 담긴 csv파일 읽어오기
# DataFrame(이하 DF) 형태로 생성
df_url = pd.read_csv('./seoul_salon_api_key.csv', encoding='cp949')
df_url.head(1)

,url,key,district
0,http://openAPI.gangnam.go.kr:8088/(인증키)/json/G...,6d7746646664646f313230436c6b7659,강남


In [3]:
# url 데이터 개수 : 25개
len(df_url)

25

In [4]:
# DF의 value(값)만 보기 
df_url.values[:1]

array([['http://openAPI.gangnam.go.kr:8088/(인증키)/json/GnPublicHygieneBizCosmeNomal/1/1/',
        '6d7746646664646f313230436c6b7659', '강남']], dtype=object)

In [5]:
# 'url' 컬럼에서 '(인증키)' 문자열을 'key' 컬럼값으로 치환하는 함수 생성
def modify_url(dataframe):
    return list(map(lambda x: re.sub('\(인증키\)', x[1] ,x[0]), dataframe.values))

In [6]:
# 치환함수 modify_url에 DF를 넣어 key값이 제대로 들어간 url을 list에 저장
# url 끝부분 두 개의 정수값은 요청하는 시작 데이터 번호 / 종료 데이터 번호

# 아직 총 데이터가 몇개인지 몰라 종료 데이터 번호를 지정할 수 없다.
# 먼저 한 개의 데이터만 불러와 총 데이터 수를 확인할 것
# 그리고 총 데이터 수를 종료 데이터 번호에 대입할 것이다.

url_list = modify_url(df_url)
url_list[:1]

['http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1/1/']

In [7]:
# 총 데이터 수 구하기 예시
# url_list의 0번째 인덱스에 저장된 강남구 url
url_list[0]

'http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1/1/'

In [8]:
# requests 모듈의 get()함수롤 사용하여 json 형태의 데이터를 가져온다.
# 여기서 "list_total_count" 에 대응하는 값이 전체 데이터 수
# 이 값들을 뽑아 요청해야 할 종료 데이터 번호로 지정해줄 것이다.

requests.get(url_list[0]).text

'{"GnPublicHygieneBizCosmeNomal":{"list_total_count":1785,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"CGG_CODE":"3220000","SNT_COB_CODE":"211","YY":"1983","UPSO_SNO":"00001","SNT_COB_NM":"미용업(일반)","UPSO_GSL_YMD":"19830627","UPSO_NM":"백민재헤어샵","TRDP_AREA":19.8,"UPSO_SITE_TELNO":"02 5451290","BMAN_STDT":"20110502","BUP_NM":"","SITE_STDT":"20180515","ADMDNG_NM":"청담동","DCB_YMD":"","DCB_WHY":"","SNT_UPTAE_NM":"일반미용업","ED_FIN_YMD":"20110622","GAEKSIL":0.0,"HANSHIL":0.0,"YANGSIL":0.0,"CHAIR_NUM":3.0,"YOKSIL":0.0,"BALHANSIL_YN":"N","WASHMC_NUM":0.0,"PERM_NT_NO":"3220000-211-1983-00001","KOR_FRGNR_GBN":"내국인","NTN":"","SITE_ADDR_RD":"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)","SITE_ADDR":"서울특별시 강남구 청담동  133번지 3호  화천회관빌딩"}]}}'

In [9]:
# url_list의 url을 하나씩 requests.get()하여 json 형태로 반환
# json_list 객체에 저장
json_list = list(map(lambda x: requests.get(x).text, url_list))

In [10]:
# json_list 확인
json_list[:2]

['{"GnPublicHygieneBizCosmeNomal":{"list_total_count":1785,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"CGG_CODE":"3220000","SNT_COB_CODE":"211","YY":"1983","UPSO_SNO":"00001","SNT_COB_NM":"미용업(일반)","UPSO_GSL_YMD":"19830627","UPSO_NM":"백민재헤어샵","TRDP_AREA":19.8,"UPSO_SITE_TELNO":"02 5451290","BMAN_STDT":"20110502","BUP_NM":"","SITE_STDT":"20180515","ADMDNG_NM":"청담동","DCB_YMD":"","DCB_WHY":"","SNT_UPTAE_NM":"일반미용업","ED_FIN_YMD":"20110622","GAEKSIL":0.0,"HANSHIL":0.0,"YANGSIL":0.0,"CHAIR_NUM":3.0,"YOKSIL":0.0,"BALHANSIL_YN":"N","WASHMC_NUM":0.0,"PERM_NT_NO":"3220000-211-1983-00001","KOR_FRGNR_GBN":"내국인","NTN":"","SITE_ADDR_RD":"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)","SITE_ADDR":"서울특별시 강남구 청담동  133번지 3호  화천회관빌딩"}]}}',
 '{"GdPublicHygieneBizCosmeNomal":{"list_total_count":927,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"CGG_CODE":"3240000","SNT_COB_CODE":"211","YY":"1979","UPSO_SNO":"00001","SNT_COB_NM":"미용업(일반)","UPSO_GSL_YMD":"19791113","UPSO_NM":"꼬꼬

In [11]:
# 25개 지역별 데이터가 잘 들어왔다.
len(json_list)

25

In [12]:
# json 파일을 json 모듈의 loads()함수를 통해 dict 객체로 변환
# dict_list에 저장
dict_list = list(map(lambda x: json.loads(x), json_list))
dict_list[:2]

[{'GnPublicHygieneBizCosmeNomal': {'list_total_count': 1785,
   'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
   'row': [{'CGG_CODE': '3220000',
     'SNT_COB_CODE': '211',
     'YY': '1983',
     'UPSO_SNO': '00001',
     'SNT_COB_NM': '미용업(일반)',
     'UPSO_GSL_YMD': '19830627',
     'UPSO_NM': '백민재헤어샵',
     'TRDP_AREA': 19.8,
     'UPSO_SITE_TELNO': '02 5451290',
     'BMAN_STDT': '20110502',
     'BUP_NM': '',
     'SITE_STDT': '20180515',
     'ADMDNG_NM': '청담동',
     'DCB_YMD': '',
     'DCB_WHY': '',
     'SNT_UPTAE_NM': '일반미용업',
     'ED_FIN_YMD': '20110622',
     'GAEKSIL': 0.0,
     'HANSHIL': 0.0,
     'YANGSIL': 0.0,
     'CHAIR_NUM': 3.0,
     'YOKSIL': 0.0,
     'BALHANSIL_YN': 'N',
     'WASHMC_NUM': 0.0,
     'PERM_NT_NO': '3220000-211-1983-00001',
     'KOR_FRGNR_GBN': '내국인',
     'NTN': '',
     'SITE_ADDR_RD': '서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)',
     'SITE_ADDR': '서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'}]}},
 {'GdPublicHygieneBizCosmeNomal': {'list_to

In [13]:
# dict 객체에서 'list_total_count'에 대응하는 총 데이터 수만 추출
# total_count_list에 저장
# 단 여기서 int 객체를 string 객체로 변환하여 저장한다.

total_count_list = list(map(lambda x: str(list(x.values())[0]['list_total_count']), dict_list))
total_count_list

['1785',
 '927',
 '1254',
 '1835',
 '1514',
 '822',
 '1309',
 '346',
 '818',
 '1073',
 '1026',
 '1206',
 '1434',
 '1031',
 '927',
 '765',
 '1357',
 '2051',
 '1325',
 '1205',
 '686',
 '1750',
 '262',
 '513',
 '938']

In [14]:
# 요청할 데이터 시작번호, 끝번호를 지정할 함수 정의
# 1회 요청 최대 데이터 건수는 1,000건이므로
# 1,000건이 넘을 시 분할 해주어야함
# ('시작번호', '끝번호') 형태의 tuple로 반환

def page_range(c_number):
    val='',
    if len(c_number) < 4: # 총 데이터 수가 1,000 미만인 경우
        val = ('1', c_number),
    else: # 1,000 이상인 경우
        if c_number == '1000': # 1,000건까지 조회가 가능하다
            val = ('1', c_number),
        elif c_number[0] == '1':
            val = (('1', '1000'), ('1001', c_number))
        elif c_number[0] == '2':
            val =  (('1', '1000'), ('1001', '2000'), ('2001', c_number))
        elif c_number[0] == '3':
            val = (('1', '1000'), ('1001', '2000'), ('2001', '3000'), ('3001', c_number))
    return val

In [15]:
# 총 데이터 수를 정의한 page_range 함수에 넣어 요청할 데이터 시작, 끝 번호 추출
# page_list에 저장

page_list = list(map(lambda x: page_range(x), total_count_list))
page_list

[(('1', '1000'), ('1001', '1785')),
 (('1', '927'),),
 (('1', '1000'), ('1001', '1254')),
 (('1', '1000'), ('1001', '1835')),
 (('1', '1000'), ('1001', '1514')),
 (('1', '822'),),
 (('1', '1000'), ('1001', '1309')),
 (('1', '346'),),
 (('1', '818'),),
 (('1', '1000'), ('1001', '1073')),
 (('1', '1000'), ('1001', '1026')),
 (('1', '1000'), ('1001', '1206')),
 (('1', '1000'), ('1001', '1434')),
 (('1', '1000'), ('1001', '1031')),
 (('1', '927'),),
 (('1', '765'),),
 (('1', '1000'), ('1001', '1357')),
 (('1', '1000'), ('1001', '2000'), ('2001', '2051')),
 (('1', '1000'), ('1001', '1325')),
 (('1', '1000'), ('1001', '1205')),
 (('1', '686'),),
 (('1', '1000'), ('1001', '1750')),
 (('1', '262'),),
 (('1', '513'),),
 (('1', '938'),)]

In [16]:
# url에서 시작번호와 끝번호에 값을 대입하기 위해
# re.split()함수로 문자열을 잘라준다.
# split_url 에 저장

split_url = list(map(lambda x: re.split('/',x), url_list))
split_url[:1]

[['http:',
  '',
  'openAPI.gangnam.go.kr:8088',
  '6d7746646664646f313230436c6b7659',
  'json',
  'GnPublicHygieneBizCosmeNomal',
  '1',
  '1',
  '']]

In [17]:
# 잘라진 split_url과 페이지 시작.끝번호 page_list를 zip()함수로 묶어준다.
combined_list = list(zip(split_url, page_list))
combined_list[:4]

[(['http:',
   '',
   'openAPI.gangnam.go.kr:8088',
   '6d7746646664646f313230436c6b7659',
   'json',
   'GnPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '1000'), ('1001', '1785'))),
 (['http:',
   '',
   'openAPI.gd.go.kr:8088',
   '744478444564646f31313163734a674d',
   'json',
   'GdPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '927'),)),
 (['http:',
   '',
   'openAPI.gangbuk.go.kr:8088',
   '414870515564646f38335946756a78',
   'json',
   'GbPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '1000'), ('1001', '1254'))),
 (['http:',
   '',
   'openAPI.gangseo.seoul.kr:8088',
   '586753714764646f3930534369636b',
   'json',
   'GangseoPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '1000'), ('1001', '1835')))]

In [18]:
# df_url의 'district' 컬럼 값들을 list 형태로 저장
district_list = list(df_url['district'].values)
district_list

['강남',
 '강동',
 '강북',
 '강서',
 '관악',
 '광진',
 '구로',
 '금천',
 '노원',
 '도봉',
 '동대문',
 '동작',
 '마포',
 '서대문',
 '서초',
 '성동',
 '성북',
 '송파',
 '양천',
 '영등포',
 '용산',
 '은평',
 '종로',
 '중구',
 '중랑']

In [19]:
# 시작번호와 끝번호에 각각의 값을 대입
# district_list를 이용해 지역명과 함께 tuple로 묶어서 저장한다.
# 완성된 최종 url을 final_url에 저장한다.
final_url=[]
for i, v in enumerate(combined_list):
    for page in v[1]:
        v[0][-3] = page[0]
        v[0][-2] = page[1]
        final_url.append(('/'.join(v[0]), district_list[i]))

In [20]:
# final_url 확인
final_url[:3]

[('http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1/1000/',
  '강남'),
 ('http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1001/1785/',
  '강남'),
 ('http://openAPI.gd.go.kr:8088/744478444564646f31313163734a674d/json/GdPublicHygieneBizCosmeNomal/1/927/',
  '강동')]

In [21]:
# 최종 url 개수 확인
# 요청할 회수 / requests.get()함수로 데이터를 읽을 회수와 같다.
len(final_url)

41

---
### 완성된 url로 이제 전체 데이터를 조회해보자!

In [22]:
# 최종 완성된 url로 다시 json파일을 요청
json_list = list(map(lambda x: (x[1], requests.get(x[0]).text), final_url))

In [23]:
# 요청하여 받은 json 파일을 dict 객체로 변환
dict_list = list(map(lambda x: (x[0], json.loads(x[1])), json_list))

In [24]:
# 에러 없이 데이터 조회가 되었는지 확인 - 'RESULT' key로 확인
error_dict = list(map(lambda x: list(x[1].values())[0]['RESULT'], dict_list))
DataFrame(error_dict)

,CODE,MESSAGE
0,INFO-000,정상 처리되었습니다
1,INFO-000,정상 처리되었습니다
2,INFO-000,정상 처리되었습니다
3,INFO-000,정상 처리되었습니다
4,INFO-000,정상 처리되었습니다
5,INFO-000,정상 처리되었습니다
6,INFO-000,정상 처리되었습니다
7,INFO-000,정상 처리되었습니다
8,INFO-000,정상 처리되었습니다
9,INFO-000,정상 처리되었습니다


In [25]:
# 41번 모두 에러없이 조회되었다.
# 에러 발생시 다음 정보를 확인할 것
pd.read_csv('./seoul_salon_error_code.csv', encoding='cp949')

,code,state
0,INFO-000,정상 처리되었습니다
1,ERROR-300,필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.
2,INFO-100,"인증키가 유효하지 않습니다. 인증키가 없는 경우, 열린 데이터 광장 홈페이지에서 ..."
3,ERROR-301,파일타입 값이 누락 혹은 유효하지 않습니다. 요청인자 중 TYPE을 확인하십시오.
4,ERROR-310,해당하는 서비스를 찾을 수 없습니다. 요청인자 중 SERVICE를 확인하십시오.
5,ERROR-331,요청시작위치 값을 확인하십시오. 요청인자 중 START_INDEX를 확인하십시오.
6,ERROR-332,요청종료위치 값을 확인하십시오. 요청인자 중 END_INDEX를 확인하십시오.
7,ERROR-333,요청위치 값의 타입이 유효하지 않습니다. 요청위치 값은 정수를 입력하세요.
8,ERROR-334,요청종료위치 보다 요청시작위치가 더 큽니다. 요청시작조회건수는 정수를 입력하세요.
9,ERROR-335,샘플데이터(샘플키:sample) 는 한번에 최대 5건을 넘을 수 없습니다. 요청시...


In [26]:
# 필요한 데이터는 'row' key에 대응하는 value이다.
# 조회 예시
list(dict_list[0][1].values())[0]['row'][:2]

[{'CGG_CODE': '3220000',
  'SNT_COB_CODE': '211',
  'YY': '1983',
  'UPSO_SNO': '00001',
  'SNT_COB_NM': '미용업(일반)',
  'UPSO_GSL_YMD': '19830627',
  'UPSO_NM': '백민재헤어샵',
  'TRDP_AREA': 19.8,
  'UPSO_SITE_TELNO': '02 5451290',
  'BMAN_STDT': '20110502',
  'BUP_NM': '',
  'SITE_STDT': '20180515',
  'ADMDNG_NM': '청담동',
  'DCB_YMD': '',
  'DCB_WHY': '',
  'SNT_UPTAE_NM': '일반미용업',
  'ED_FIN_YMD': '20110622',
  'GAEKSIL': 0.0,
  'HANSHIL': 0.0,
  'YANGSIL': 0.0,
  'CHAIR_NUM': 3.0,
  'YOKSIL': 0.0,
  'BALHANSIL_YN': 'N',
  'WASHMC_NUM': 0.0,
  'PERM_NT_NO': '3220000-211-1983-00001',
  'KOR_FRGNR_GBN': '내국인',
  'NTN': '',
  'SITE_ADDR_RD': '서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)',
  'SITE_ADDR': '서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'},
 {'CGG_CODE': '3220000',
  'SNT_COB_CODE': '211',
  'YY': '1983',
  'UPSO_SNO': '00001',
  'SNT_COB_NM': '미용업(일반)',
  'UPSO_GSL_YMD': '19830627',
  'UPSO_NM': '백민재헤어샵',
  'TRDP_AREA': 65.67,
  'UPSO_SITE_TELNO': '02 5451290',
  'BMAN_STDT': '20110502',
  'BUP_N

In [27]:
# 조회한 데이터에서 필요한 'row' key에 해당하는 값만 뽑아오기
real_dict = list(map(lambda x: list(x[1].values())[0]['row'], dict_list))

In [28]:
# 조회한 데이터에서 지역명만 따로 뽑아 district_list에 다시 저장
# 요청 페이지를 분할한 경우 같은 지역명이 여러번 찍힌다.

district_list = list(map(lambda x: x[0], dict_list))
district_list

['강남',
 '강남',
 '강동',
 '강북',
 '강북',
 '강서',
 '강서',
 '관악',
 '관악',
 '광진',
 '구로',
 '구로',
 '금천',
 '노원',
 '도봉',
 '도봉',
 '동대문',
 '동대문',
 '동작',
 '동작',
 '마포',
 '마포',
 '서대문',
 '서대문',
 '서초',
 '성동',
 '성북',
 '성북',
 '송파',
 '송파',
 '송파',
 '양천',
 '양천',
 '영등포',
 '영등포',
 '용산',
 '은평',
 '은평',
 '종로',
 '중구',
 '중랑']

In [29]:
# dict 객체를 DF로 변환
# df_list에 저장
df_list = list(map(lambda x: DataFrame(x), real_dict))

In [30]:
# df_list 조회 - 0번째 DF는 '강남구' 미용실 정보
df_list[0].head(2)

,CGG_CODE,SNT_COB_CODE,YY,UPSO_SNO,SNT_COB_NM,UPSO_GSL_YMD,UPSO_NM,TRDP_AREA,UPSO_SITE_TELNO,BMAN_STDT,BUP_NM,SITE_STDT,ADMDNG_NM,DCB_YMD,DCB_WHY,SNT_UPTAE_NM,ED_FIN_YMD,GAEKSIL,HANSHIL,YANGSIL,CHAIR_NUM,YOKSIL,BALHANSIL_YN,WASHMC_NUM,PERM_NT_NO,KOR_FRGNR_GBN,NTN,SITE_ADDR_RD,SITE_ADDR
0,3220000,211,1983,00001,미용업(일반),19830627,백민재헤어샵,19.80,02 5451290,20110502,,20180515,청담동,,,일반미용업,20110622,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,3220000,211,1983,00001,미용업(일반),19830627,백민재헤어샵,65.67,02 5451290,20110502,,20180515,청담동,,,일반미용업,20110622,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [31]:
# df_list에 저장된 모든 DF에 '지역' 상위 인덱스 추가하기
for i, df in enumerate(df_list):
    df_list[i].index = [[district_list[i]]*len(df_list[i]), df_list[i].index]

In [32]:
# 계층형 인덱스의 상위 인덱스에 지역명이 잘 들어갔는지 확인
df_list[0].head(2)

CGG_CODE SNT_COB_CODE    YY UPSO_SNO SNT_COB_NM UPSO_GSL_YMD UPSO_NM  \
강남 0  3220000          211  1983    00001    미용업(일반)     19830627  백민재헤어샵   
   1  3220000          211  1983    00001    미용업(일반)     19830627  백민재헤어샵   

      TRDP_AREA UPSO_SITE_TELNO BMAN_STDT BUP_NM SITE_STDT ADMDNG_NM DCB_YMD  \
강남 0      19.80      02 5451290  20110502         20180515       청담동           
   1      65.67      02 5451290  20110502         20180515       청담동           

     DCB_WHY SNT_UPTAE_NM ED_FIN_YMD  GAEKSIL  HANSHIL  YANGSIL  CHAIR_NUM  \
강남 0                일반미용업   20110622      0.0      0.0      0.0        3.0   
   1                일반미용업   20110622      0.0      0.0      0.0        3.0   

      YOKSIL BALHANSIL_YN  WASHMC_NUM              PERM_NT_NO KOR_FRGNR_GBN  \
강남 0     0.0            N         0.0  3220000-211-1983-00001           내국인   
   1     0.0            N         0.0  3220000-211-1983-00001           내국인   

     NTN                                  SITE_ADDR_RD  \
강남 0      서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1      서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   

                            SITE_ADDR  
강남 0  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  
   1  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩

In [33]:
# DF 하나로 병합
df_all = pd.concat(df_list)

In [34]:
# 병합된 DF 확인
# 마지막 중랑구 데이터까지 잘 들어간 모습
df_all.tail(3)

CGG_CODE SNT_COB_CODE    YY UPSO_SNO SNT_COB_NM UPSO_GSL_YMD  \
중랑 935  3060000          211  2020    00008    미용업(일반)     20200217   
   936  3060000          211  2020    00009    미용업(일반)     20200220   
   937  3060000          211  2020    00010    미용업(일반)     20200310   

           UPSO_NM  TRDP_AREA UPSO_SITE_TELNO BMAN_STDT BUP_NM SITE_STDT  \
중랑 935  에이데이즈 헤어살롱      90.41                  20200217         20200217   
   936    지노헤어 상봉점     163.05                  20200220         20200220   
   937        우주헤어      19.65                  20200310         20200310   

       ADMDNG_NM DCB_YMD DCB_WHY SNT_UPTAE_NM ED_FIN_YMD  GAEKSIL  HANSHIL  \
중랑 935      신내1동                        일반미용업                 0.0      0.0   
   936     상봉제2동                        일반미용업                 0.0      0.0   
   937     상봉제1동                        일반미용업                 0.0      0.0   

        YANGSIL  CHAIR_NUM  YOKSIL BALHANSIL_YN  WASHMC_NUM  \
중랑 935      0.0        5.0     0.0            N         0.0   
   936      0.0        8.0     0.0            N         0.0   
   937      0.0        4.0     0.0            N         0.0   

                    PERM_NT_NO KOR_FRGNR_GBN NTN  \
중랑 935  3060000-211-2020-00008           내국인       
   936  3060000-211-2020-00009           내국인       
   937  3060000-211-2020-00010           내국인       

                            SITE_ADDR_RD                   SITE_ADDR  
중랑 935      서울특별시 중랑구 봉화산로 232, 2층 (신내동)   서울특별시 중랑구 신내동  405번지 5호    
   936        서울특별시 중랑구 망우로 282-1, (상봉동)  서울특별시 중랑구 상봉동  115번지 14호    
   937  서울특별시 중랑구 망우로43길 3, 1층 좌측호 (상봉동)  서울특별시 중랑구 상봉동  116번지 45호

In [35]:
# 인덱스에 인덱스명 주기
df_all.index.names=['지역', '순번']
df_all.head(2)

CGG_CODE SNT_COB_CODE    YY UPSO_SNO SNT_COB_NM UPSO_GSL_YMD UPSO_NM  \
지역 순번                                                                        
강남 0   3220000          211  1983    00001    미용업(일반)     19830627  백민재헤어샵   
   1   3220000          211  1983    00001    미용업(일반)     19830627  백민재헤어샵   

       TRDP_AREA UPSO_SITE_TELNO BMAN_STDT BUP_NM SITE_STDT ADMDNG_NM DCB_YMD  \
지역 순번                                                                           
강남 0       19.80      02 5451290  20110502         20180515       청담동           
   1       65.67      02 5451290  20110502         20180515       청담동           

      DCB_WHY SNT_UPTAE_NM ED_FIN_YMD  GAEKSIL  HANSHIL  YANGSIL  CHAIR_NUM  \
지역 순번                                                                         
강남 0                 일반미용업   20110622      0.0      0.0      0.0        3.0   
   1                 일반미용업   20110622      0.0      0.0      0.0        3.0   

       YOKSIL BALHANSIL_YN  WASHMC_NUM              PERM_NT_NO KOR_FRGNR_GBN  \
지역 순번                                                                          
강남 0      0.0            N         0.0  3220000-211-1983-00001           내국인   
   1      0.0            N         0.0  3220000-211-1983-00001           내국인   

      NTN                                  SITE_ADDR_RD  \
지역 순번                                                     
강남 0       서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1       서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   

                             SITE_ADDR  
지역 순번                                   
강남 0   서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  
   1   서울특별시 강남구 청담동  133번지 3호  화천회관빌딩

---
### DF를 csv 파일로 저장

In [36]:
# 오늘 날짜 뽑기
# time모듈로 날짜를 불러와 파일명 뒤에 붙여줄 건데 기본적으로 '월/일/년' 형태로 뽑힌다.
# '/' 문자는 파일명에 쓸 수 없기 때문에 '-' 문자로 바꿔준다.

date = time.strftime('%x', time.localtime(time.time()))
date = date.replace('/','-')
date

'03-26-20'

In [37]:
# 파일 이름 지정
file_name = './seoul_salon_' + date + ".csv"
file_name

'./seoul_salon_03-26-20.csv'

In [38]:
# csv 파일로 저장하기
df_all.to_csv(file_name, encoding='cp949')